In [24]:
# IMPORT REQUIRED LIBRARIES
# YOU SHOULD MAKE SURE THAT YOU ALREADY INSTLALED THEM PREVIOUSLY
import yfinance as yf
import finplot as fplt
# chamada_api = yf.Ticker('PETR4.SA').history(period='36mo')
# Create a finplot instance
chamada_api = yf.Ticker('PETR4.SA').history(period='1mo')
b = fplt.background = fplt.odd_plot_background = '#242320'
w = fplt.foreground = '#eef'
fplt.cross_hair_color = w+'a'
ax, axv = fplt.create_plot(title='Gráfico de Velas', rows = 2, maximize=True)
# Plot the data on the graph
candles = chamada_api[['Open', 'Close', 'High', 'Low']]
daily_plot = fplt.candlestick_ochl(candles, candle_width=1)
daily_plot.colors.update(dict(bull_body='#bfb', bull_shadow='#ada', bear_body='#fbc', bear_shadow='#dab'))

# Add some additional features to the plot

fplt.show()

In [ ]:
# IMPORT REQUIRED LIBRARIES
# YOU SHOULD MAKE SURE THAT YOU ALREADY INSTLALED THEM PREVIOUSLY
import yfinance as yf
import finplot as fplt
# chamada_api = yf.Ticker('PETR4.SA').history(period='36mo')
# Create a finplot instance
chamada_api = yf.Ticker('PETR4.SA').history(period='36mo')
b = fplt.background = fplt.odd_plot_background = '#242320'
w = fplt.foreground = '#eef'
fplt.cross_hair_color = w+'a'
ax, axv = fplt.create_plot(title='Gráfico de Velas', rows = 2, maximize=False)
# Plot the data on the graph
candles = chamada_api[['Open', 'Close', 'High', 'Low']]
daily_plot = fplt.candlestick_ochl(candles, candle_width=1)
daily_plot.colors.update(dict(bull_body='#bfb', bull_shadow='#ada', bear_body='#fbc', bear_shadow='#dab'))
df_renko = chamada_api.reset_index()
fplt.volume_ocv(df_renko[['Date','Open','Close','Volume']], ax=axv, )
# Add some additional features to the plot

fplt.show()

In [ ]:
# IMPORT REQUIRED LIBRARIES
# YOU SHOULD MAKE SURE THAT YOU ALREADY INSTLALED THEM PREVIOUSLY
import yfinance as yf
import finplot as fplt
# chamada_api = yf.Ticker('PETR4.SA').history(period='36mo')
# Create a finplot instance
chamada_api = yf.Ticker('PETR4.SA').history(period='36mo')
b = fplt.background = fplt.odd_plot_background = '#242320'
w = fplt.foreground = '#eef'
fplt.cross_hair_color = w+'a'
ax, axv = fplt.create_plot(title='Gráfico de Velas', rows = 2, maximize=False)
# Plot the data on the graph
candles = chamada_api[['Open', 'Close', 'High', 'Low']]
daily_plot = fplt.candlestick_ochl(candles, candle_width=1)
daily_plot.colors.update(dict(bull_body='#bfb', bull_shadow='#ada', bear_body='#fbc', bear_shadow='#dab'))
df_renko = chamada_api.reset_index()
fplt.volume_ocv(df_renko[['Date','Open','Close','Volume']], ax=axv, )
# Add some additional features to the plot
mean_volume = chamada_api['Volume'].mean()
fplt.add_line(axv,mean_volume, color='#4f4f4f', style='--')
fplt.show()

In [1]:
# IMPORT REQUIRED LIBRARIES
# YOU SHOULD MAKE SURE THAT YOU ALREADY INSTLALED THEM PREVIOUSLY
import yfinance as yf
import finplot as fplt
# chamada_api = yf.Ticker('PETR4.SA').history(period='36mo')
# Create a finplot instance
chamada_api = yf.Ticker('PETR4.SA').history(period='5mo')
b = fplt.background = fplt.odd_plot_background = '#242320'
w = fplt.foreground = '#eef'
fplt.cross_hair_color = w+'a'
ax, axv = fplt.create_plot(title='Gráfico de Velas', rows = 2, maximize=False)
# Plot the data on the graph
candles = chamada_api[['Open', 'Close', 'High', 'Low']]
daily_plot = fplt.candlestick_ochl(candles, candle_width=1)
daily_plot.colors.update(dict(bull_body='#bfb', bull_shadow='#ada', bear_body='#fbc', bear_shadow='#dab'))
df_renko = chamada_api.reset_index()
fplt.volume_ocv(df_renko[['Date','Open','Close','Volume']], ax=axv, )
# Add some additional features to the plot
fplt.plot(df_renko.Volume.ewm(span=24).mean(), ax=axv, color=1)
fplt.show()

In [26]:
# IMPORT REQUIRED LIBRARIES
# YOU SHOULD MAKE SURE THAT YOU ALREADY INSTLALED THEM PREVIOUSLY
import yfinance as yf
import finplot as fplt
# chamada_api = yf.Ticker('PETR4.SA').history(period='36mo')
# Create a finplot instance
chamada_api = yf.Ticker('PETR4.SA').history(period='5mo')
from collections import defaultdict
import dateutil.parser
import finplot as fplt
import numpy as np
import pandas as pd
import requests


#baseurl = 'https://www.bitmex.com/api'





def plot_accumulation_distribution(df, ax):
    ad = (2*df.Close-df.High-df.Low) * df.Volume / (df.High - df.Low)
    ad.cumsum().ffill().plot(ax=ax, legend='Accum/Dist', color='#f00000')


def plot_bollinger_bands(df, ax):
    mean = df.Close.rolling(20).mean()
    stddev = df.Close.rolling(20).std()
    df['boll_hi'] = mean + 2.5*stddev
    df['boll_lo'] = mean - 2.5*stddev
    p0 = df.boll_hi.plot(ax=ax, color='#808080', legend='BB')
    p1 = df.boll_lo.plot(ax=ax, color='#808080')
    fplt.fill_between(p0, p1, color='#1A1C1D')


def plot_ema(df, ax):
    df.Close.ewm(span=9).mean().plot(ax=ax, legend='EMA',color = '#eef')


def plot_heikin_ashi(df, ax):
    df['h_close'] = (df.Open+df.Close+df.High+df.Low) / 4
    ho = (df.Open.iloc[0] + df.Close.iloc[0]) / 2
    for i,hc in zip(df.index, df['h_close']):
        df.loc[i, 'h_open'] = ho
        ho = (ho + hc) / 2
    print(df['h_open'])
    df['h_high'] = df[['High','h_open','h_close']].max(axis=1)
    df['h_low'] = df[['Low','h_open','h_close']].min(axis=1)
    df[['h_open','h_close','h_high','h_low']].plot(ax=ax, kind='candle')


def plot_heikin_ashi_volume(df, ax):
    df[['h_open','h_close','Volume']].plot(ax=ax, kind='volume')


def plot_rsi(df, ax):
    diff = df.Close.diff().values
    gains = diff
    losses = -diff
    with np.errstate(invalid='ignore'):
        gains[(gains<0)|np.isnan(gains)] = 0.0
        losses[(losses<=0)|np.isnan(losses)] = 1e-10 # we don't want divide by zero/NaN
    n = 14
    m = (n-1) / n
    ni = 1 / n
    g = gains[n] = np.nanmean(gains[:n])
    l = losses[n] = np.nanmean(losses[:n])
    gains[:n] = losses[:n] = np.nan
    for i,v in enumerate(gains[n:],n):
        g = gains[i] = ni*v + m*g
    for i,v in enumerate(losses[n:],n):
        l = losses[i] = ni*v + m*l
    rs = gains / losses
    df['rsi'] = 100 - (100/(1+rs))
    df.rsi.plot(ax=ax, legend='RSI')
    fplt.set_y_range(0, 100, ax=ax)
    fplt.add_band(3, 97, ax=ax, color='#1A1C1D')


def plot_vma(df, ax):
    df.Volume.rolling(20).mean().plot(ax=ax, color='#c0c030')



ax,axv,ax2,ax4 = fplt.create_plot(title='Gráfico de Velas', rows = 4, maximize=True)

# price chart
#plot_heikin_ashi(chamada_api, ax)
plot_bollinger_bands(chamada_api, ax)
plot_ema(chamada_api, ax)

# volume chart
#plot_heikin_ashi_volume(chamada_api, axv)
#plot_vma(chamada_api, ax=axv)

# some more charts
plot_accumulation_distribution(chamada_api, ax2)
plot_rsi(chamada_api, ax4)




b = fplt.background = fplt.odd_plot_background = '#010101'
w = fplt.foreground = '#eef'
fplt.cross_hair_color = w+'a'
# Plot the data on the graph
candles = chamada_api[['Open', 'Close', 'High', 'Low']]
daily_plot = fplt.candlestick_ochl(candles, candle_width=1)
daily_plot.colors.update(dict(bull_body='#bfb', bull_shadow='#ada', bear_body='#fbc', bear_shadow='#dab'))
df_renko = chamada_api.reset_index()
fplt.volume_ocv(df_renko[['Date','Open','Close','Volume']], ax=axv, )
# Add some additional features to the plot
fplt.plot(df_renko.Volume.ewm(span=24).mean(), ax=axv, color='#eef')
fplt.show()
